In [1]:
!pip install pandas pyarrow fastparquet

In [2]:
import pandas as pd

df = pd.read_parquet("hf://datasets/KAIST-IC-LAB721/SDCNL/data/train-00000-of-00001.parquet")

c:\Users\Sabrina Lopez\anaconda3\envs\nlpproject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!pip install huggingface_hub

In [4]:
df

,title,text,label,label_text
0,Need help,Hi I don't really know how to phrase this situ...,0,depression
1,feeling so overwhelmed and hopeless,i have been so depressed these past couple wee...,1,suicidal
2,"Nothing matters anymore, getting worse",Hi..I don't know where else to go. I am devast...,0,depression
3,Who’s tired of hearing bullshit,"The shit like “it will get better, everyone is...",1,suicidal
4,I wish I was someone else.,I wish I was prettier. I wish I didn’t feel li...,0,depression
...,...,...,...,...
1890,think its over,i just don’t wanna live anymore so yeah,0,depression
1891,To all of those feeling isolated and suffering...,I’ve learned that life is fucking sad sometime...,0,depression
1892,I just really wish I had died the first time I...,That's all. Nothing has gotten better and I've...,1,suicidal
1893,I feel unimportant.,Not the first time I'm going through this of c...,0,depression


In [5]:
df['text'].head()

0    Hi I don't really know how to phrase this situ...
1    i have been so depressed these past couple wee...
2    Hi..I don't know where else to go. I am devast...
3    The shit like “it will get better, everyone is...
4    I wish I was prettier. I wish I didn’t feel li...
Name: text, dtype: object

In [6]:
for idx, str in enumerate(df['text']):
    # print(string)

    clean_str = list([val for val in str if val.isalnum() or val == ' '])
    clean_str = ''.join(clean_str)
    low_clean_str = clean_str.lower()

    # print(low_clean_str)

    df = df.replace(df['text'][idx], low_clean_str)

In [7]:
df['text'][0]

'hi i dont really know how to phrase this situation but ill try my life is at a really good point right now im never really depressed over stuff and 99 percent of the time my mind is clear im about to graduate high school and im really excited however people in my family and friend group have tons of issues wether they sleep all day hate themselves or have no ambition to keep living on in this world theyve got problems i dont wanna sound like im gloating but usually im the person that a lot of these people end up going to because usually im able to talk people through issues and help them in the long run yeah sometimes their issues make me really sad and stuff because who doesnt feel sad when people are telling you they feel worthlessbut today one of my best friends showed me that he was cutting and it really effected me i talked to him about it and its mostly because of how painfully bored he is and he doesnt even know why hes doing it hes a pretty logical guy he will go to class and 

In [8]:
!pip install transformers sentencepiece torch

In [9]:
from transformers import AutoTokenizer, XLMRobertaModel

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaModel.from_pretrained('xlm-roberta-base')

In [10]:
!pip install tensorflow

In [14]:
example = tokenizer(['Hello World', 'hi how are you'], padding=True, truncation=True, return_tensors='pt')
example

{'input_ids': tensor([[    0, 35378,  6661,     2,     1,     1],
        [    0,  1274,  3642,   621,   398,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1]])}

In [15]:
!pip install torch

In [16]:
import torch

example['input_ids'] = example['input_ids'].type(torch.LongTensor) 
testing = model(**example)
testing

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 1.0244e-01,  1.2357e-01,  6.2832e-02,  ..., -1.2062e-01,
           4.5713e-02,  3.6954e-03],
         [-3.0482e-02,  5.7443e-02,  1.5464e-02,  ...,  2.3387e-02,
          -5.8822e-02,  1.5089e-02],
         [ 6.9837e-02,  2.5044e-02,  4.0708e-02,  ..., -2.2505e-01,
           4.4505e-03,  6.8066e-03],
         [ 9.0492e-02,  1.0865e-01, -1.1178e-02,  ..., -2.2825e-01,
          -4.2947e-02,  5.8354e-02],
         [-1.2323e-02,  9.0776e-02,  6.3362e-02,  ..., -9.4688e-02,
          -3.6719e-03, -7.3051e-02],
         [-1.2323e-02,  9.0776e-02,  6.3362e-02,  ..., -9.4688e-02,
          -3.6719e-03, -7.3051e-02]],

        [[ 7.0207e-02,  1.0434e-01,  5.7032e-02,  ..., -9.3063e-02,
           6.1594e-02,  2.3045e-02],
         [-2.5635e-02,  5.2697e-02, -4.6106e-02,  ...,  1.0901e-01,
          -5.6282e-02,  1.0096e-01],
         [-3.9276e-02,  6.5958e-02, -4.0160e-02,  ...,  3.8406e-02,
          -7.5012e-03,  6.3

In [18]:
!pip install scikit-learn

  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ----------------- ---------------------- 4.7/11.0 MB 25.9 MB/s eta 0:00:01
   ------------------------------------ --- 10.0/11.0 MB 25.9 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 24.6 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl (46.2 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [19]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)  # 80% train, 20% temp
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42) # Split temp into 50% val and 50% test


print("Train set size:", len(train_df))
print("Validation set size:", len(val_df))
print("Test set size:", len(test_df))

Train set size: 1516
Validation set size: 189
Test set size: 190


In [21]:
train_df

,title,text,label,label_text
339,So Lonely And Depressed,ever since my break up im so lonely and depres...,0,depression
1386,I feel so conflicted,i feel horrible im too scared to open up to an...,1,suicidal
544,And I am fucking FED UP with people telling me...,it will get better for you the person with mor...,1,suicidal
937,Bump in my Journey,been dealing with depression for a while but h...,0,depression
741,I'm a mess right now,i have no one to listen to my problems so i th...,0,depression
...,...,...,...,...
1130,What’s even worse than not being able to get i...,i cant believe ive driven a beloved friend to ...,1,suicidal
1294,"I turn 19 in may, I have no boyfriend/girlfrie...",i have been bullied and told i was playing the...,1,suicidal
860,Everything and nothing,thats what wrong i know im in pretty deep depr...,0,depression
1459,I want to sleep forever and never want to wake...,emptypost,1,suicidal
